In [ ]:
import pandas as pd
import numpy as np
from dateutil import relativedelta
pd.options.display.max_rows = 100
from datetime import datetime
import re
from itertools import combinations, product

In [1]:
from os import pardir, path
import sys
mod_path = path.abspath(path.join(pardir))

print(mod_path)

if mod_path not in sys.path:
    sys.path.append(mod_path)

print(sys.path)
from data_extract import client, fin, company , companyfinancials_list, pnl, bs, cf, profile, prices, fx, mrktcap, prices_n_fin, idx_price, MONGO_FIN
from mypackage import calperiod, Feature, Model

/Users/karimkhalil/Coding/development/stock_portfolio
['/Users/karimkhalil/Coding/development/stock_portfolio/notebooks', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python39.zip', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python3.9', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python3.9/lib-dynload', '', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python3.9/site-packages', '/Users/karimkhalil/Coding/development/stock_portfolio']
['admin', 'config', 'db_fin', 'local']
['cf', 'prices_bckup', 'company_today', 'mrktcap', 'company_wfinancials', 'prices_n_fin', 'bs', 'fx', 'profile', 'idx_price', 'prices', 'idx_all', 'pnl']
mypackage imported


In [ ]:
db = MONGO_FIN()

In [ ]:
import json
mapping = json.load(open("../config/config.json"))['mapping']

#### Currency data

In [ ]:
df_fx1 = db.get_df(fx)
df_fx2 = df_fx1[['symbol' , 'adjClose', 'date']]
df_fx2['datetime'] = pd.to_datetime(df_fx2.date, format='%Y-%m-%d')
df_fx2['yearmonth'] = df_fx2['datetime'].apply(lambda x: x.strftime('%Y%m'))
df_fx2['year'] = df_fx2['datetime'].apply(lambda x: x.strftime('%Y'))
df_fx2['month'] = df_fx2['datetime'].apply(lambda x: x.strftime('%m'))
df_fx2.symbol = df_fx2.symbol.str.replace("/USD", "")

## DataFrame for the average fx for a given month ==> to be used to convert the fx for stock prices and market cap
df_fx3 = df_fx2.groupby(['symbol' , 'yearmonth' , 'year' , 'month'])['adjClose'].mean().reset_index()
df_fx3['key_fx'] = df_fx3.symbol+"."+df_fx3.yearmonth.astype(str)

In [ ]:
## financials FX ==> calculate fx for last 3 months
df_fx4 = df_fx3
fx_periods = list(df_fx4.yearmonth.unique())
fx_periods2 = [i for i in fx_periods]
fx_periods3 = {i: [(datetime.strptime(i, "%Y%m") + relativedelta.relativedelta(months=-j)).strftime('%Y%m')for j in range(3)] for i in fx_periods2}

## DataFrame for the average fx for a given quarter ==> to be used to convert the fx for financials
df_fx4['previous_q'] = df_fx4.apply(lambda row: fx_periods3[row['yearmonth']] , axis =1)
df_fx4['previous_q'] = df_fx4.apply(lambda row: [row['symbol']+"."+ str(i) for i in row['previous_q']] , axis =1)
df_fx4['previous_q_fx'] =  df_fx4.apply(lambda row: df_fx4.loc[df_fx4['key_fx'].isin(row['previous_q']), "adjClose"].mean() , axis =1)

#### Stock data

In [ ]:
# import csv
df_stock_mrkt1 = pd.read_csv('../data/df_stock_mrkt_q.csv')

In [ ]:
# merge fx with stock data

df_stock_mrkt2 = df_stock_mrkt1.copy()
df_stock_mrkt2.currency = df_stock_mrkt2.currency.str.upper()
df_stock_mrkt2.insert(5, 'datetime' , value = pd.to_datetime(df_stock_mrkt2.date, format='%Y-%m-%d'))
df_stock_mrkt2.insert(5, column='yearmonth', value=df_stock_mrkt2['datetime'].apply(lambda x: x.strftime('%Y%m')))
df_stock_mrkt2.insert(5, column='year', value=df_stock_mrkt2['datetime'].apply(lambda x: x.strftime('%Y')))
df_stock_mrkt2.insert(5, column='month', value=df_stock_mrkt2['datetime'].apply(lambda x: x.strftime('%m')))
df_stock_mrkt2.insert(5, column='key_fx', value=df_stock_mrkt2.currency+"."+df_stock_mrkt2.yearmonth.astype(str))
df_stock_mrkt2.insert(5, column='fx_stock', value=df_stock_mrkt2.merge(df_fx3[['key_fx' , 'adjClose']], how = 'left', on = 'key_fx')['adjClose_y'])
df_stock_mrkt2.insert(5, column='fx_quarter', value=df_stock_mrkt2.merge(df_fx4[['key_fx' , 'previous_q_fx']], how = 'left', on = 'key_fx')['previous_q_fx'])
df_stock_mrkt2.loc[df_stock_mrkt2.currency == 'USD' , 'fx_stock'] = 1
df_stock_mrkt2.loc[df_stock_mrkt2.currency == 'USD' , 'fx_quarter'] = 1

In [ ]:
quarter = {
    1: "Q1", 
    2: "Q1", 
    3: "Q1", 
    4: "Q2", 
    5: "Q2", 
    6: "Q2", 
    7: "Q3", 
    8: "Q3",
    9: "Q3",
    10: "Q4", 
    11: "Q4",
    12: "Q4"
}

In [ ]:
df_stock_mrkt2.info(verbose=True, show_counts=True)

In [ ]:
# normalize common attributes from financials
df_stock_mrkt2['fin_reportedCurrency'] = df_stock_mrkt2['bs_reportedCurrency']
df_stock_mrkt2['fin_fillingDate'] = df_stock_mrkt2['bs_fillingDate']
df_stock_mrkt2['fin_acceptedDateDate'] = df_stock_mrkt2['bs_acceptedDate']
df_stock_mrkt2['fin_calendarYear'] = df_stock_mrkt2['bs_calendarYear']
df_stock_mrkt2['fin_period'] = df_stock_mrkt2['bs_period']
df_stock_mrkt2['cal_period'] = df_stock_mrkt2['month'].apply(lambda x: quarter[int(x)])
df_stock_mrkt2['cal_yearperiod'] = df_stock_mrkt2['year'].astype(str) + df_stock_mrkt2['cal_period']
df_stock_mrkt2['key_main'] = df_stock_mrkt2['symbol'] + "_" + df_stock_mrkt2['cal_yearperiod']

In [ ]:
df_aapl = df_stock_mrkt2[df_stock_mrkt2['symbol']=='AAPL']
feature=Feature(df_aapl)
feature.check_companyperiod(df_aapl,'symbol', 'year', 'cal_yearperiod', True)

In [ ]:
df_aapl[df_aapl['year'] == '2006']

In [ ]:
# industry categorization mapping
mapping = json.load(open("../config/config.json"))['mapping']
df_stock_mrkt2['industry-category'] = df_stock_mrkt2.industry.map(mapping['industry'])
df_industries = df_stock_mrkt2.groupby(['industry' , 'industry-category'])['symbol'].count().reset_index()

In [ ]:
## industry mapping checks
df_industries = df_stock_mrkt2.groupby(['symbol' , 'industry'])['adjClose'].count().reset_index().drop(columns='adjClose')
mapping = json.load(open("../config/config.json"))['mapping']
df_industry_category = df_industries.industry.value_counts().reset_index()
df_industry_category.columns = ['industry' , 'industry_count']
df_industry_category['industry_category'] = df_industry_category.industry.map(mapping['industry'])

In [ ]:
# relevant columns for data
cols_stock = json.load(open("../config/config.json"))['cols']['stock']
df_stock_mrkt3 = df_stock_mrkt2[cols_stock]

In [ ]:
df_aapl = df_stock_mrkt3[df_stock_mrkt3['symbol']=='AAPL']
feature=Feature(df_aapl)
feature.check_companyperiod(df_aapl,'symbol', 'year', 'cal_yearperiod', True)

In [ ]:
df_stock_mrkt3.info(verbose=True, show_counts=True)

In [ ]:
# remove missing data

filter_mrktcap = df_stock_mrkt3['marketCap'].notna() ## remove companies with no market capitalization 
filter_industry = df_stock_mrkt3['industry-category'].notna() ## remove some industries 
filter_fx1 = df_stock_mrkt3['fx_quarter'].notna()
filter_fx2 = df_stock_mrkt3['fx_stock'].notna()
filter_zerocash = df_stock_mrkt3['bs_cashAndShortTermInvestments'] > 0
filter_noshell  = df_stock_mrkt3['industry-category']!= 'shell' # exclude holding companies


df_stock_mrkt4 = df_stock_mrkt3.loc[filter_fx1 & filter_fx2 & filter_mrktcap & filter_industry & filter_fx1 & filter_fx2 & filter_noshell]

df_stock_mrkt4.info(verbose=True, show_counts=True)

In [ ]:
df_aapl = df_stock_mrkt4[df_stock_mrkt4['symbol']=='AAPL']
feature=Feature(df_aapl)
feature.check_companyperiod(df_aapl,'symbol', 'year', 'cal_yearperiod', True)

In [ ]:
## remove companies with more than 1 financial in a given quarter
df_stock_mrkt4.drop_duplicates(subset='key_main' , keep='last', inplace = True)
df_stock_mrkt4.info(verbose=True, show_counts=True)

In [ ]:
df_aapl = df_stock_mrkt4[df_stock_mrkt4['symbol']=='AAPL']
feature=Feature(df_aapl)
feature.check_companyperiod(df_aapl,'symbol', 'year', 'cal_yearperiod', True)

In [ ]:
# fx translation

cols_fxspot = json.load(open("../config/config.json"))['cols']['fx_stock']
cols_fxq = json.load(open("../config/config.json"))['cols']['fx_q']

def fx(df, cols, col_fx, drop=True):
    df2 = df.copy()
    for i in cols:
        loc = df.columns.get_loc(i)
        newcol = "usd_"+i
        df2.insert(loc, newcol, df[i] * df[col_fx])
        if drop==True:
            df2.drop(columns = i, inplace=True)
    return df2

df_stock_mrkt5 = df_stock_mrkt4.copy()
df_stock_mrkt6 = df_stock_mrkt4.copy()

df_stock_mrkt5 = fx(df_stock_mrkt5, cols_fxspot, 'fx_stock', False)
df_stock_mrkt5 = fx(df_stock_mrkt5, cols_fxq, 'fx_quarter', False)

df_stock_mrkt6 = fx(df_stock_mrkt6, cols_fxspot, 'fx_stock', True)
df_stock_mrkt6 = fx(df_stock_mrkt6, cols_fxq, 'fx_quarter', True)

In [ ]:
df_stock_mrkt6.info(verbose=True, show_counts=True)

In [ ]:
# df_stock_mrkt5.to_csv('../data/data_prefeatures_wlocalfx.csv')
# df_stock_mrkt6.to_csv('../data/data_prefeatures.csv')